In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [3]:
def add_df_panel(df,timestamp_col,timestamp_back):
    
    df[timestamp_col + '_A'] = df[timestamp_col] - timestamp_back
    col_names_orig = df.columns
    
    col_join1 = timestamp_col
    col_join2 = timestamp_col +'_A'
    final =  pd.merge(df, df, left_on=col_join2, right_on=col_join1)
    final = final.sort_values(by=[timestamp_col+'_x'])
    
    col_names_new = final.columns
    
    col_names_update = []
    index = 0
    for name in col_names_new:
        if index < len(col_names_orig):
            col_names_update.append(col_names_orig[index])
        else:
            col_names_update.append(name)
        index = index + 1 
    
    final.columns = col_names_update
    final = final.drop([timestamp_col+'_A_y'], axis=1)
    
    col_names_orig = df.columns[1:]
    final = final.drop(col_names_orig, axis=1)
        
    return final

In [4]:
def create_df_panel_parquet(asset_id,start,back_window,increment,timestamp_col):
    df = train_df[train_df["Asset_ID"] == asset_id]
    col_names = df.columns

    for x in range(start,(back_window+1),1):

        time_back = increment * x

        print("Round:",x, "Time Back",time_back)

        new_panel_df = add_df_panel(df[col_names].copy(),timestamp_col,time_back)
        new_panel_df.columns = new_panel_df.columns.str.replace("_y", ("_"+str(x).zfill(6)))   
        df =  pd.merge(df, new_panel_df, how='left', left_on=timestamp_col, right_on=timestamp_col)

    df = df.sort_values(by=[timestamp_col])

    filename = 'AssetID_' + str(asset_id).zfill(6) + "_Start_" + str(start).zfill(6) + "_End_" + str(back_window).zfill(6)
    df.to_parquet(filename + '.parquet.gzip',compression='gzip')
    
    return filename

In [5]:
"""
for x in range(start,back_window,file_step_size):
    print("======================================================================================================")
    print("Start",(x+1),"End",(x+file_step_size))
    save_filename = create_df_panel_parquet(asset_id,(x+1),(x+file_step_size),increment,timestamp_col)
"""

'\nfor x in range(start,back_window,file_step_size):\n    print("======================================================================================================")\n    print("Start",(x+1),"End",(x+file_step_size))\n    save_filename = create_df_panel_parquet(asset_id,(x+1),(x+file_step_size),increment,timestamp_col)\n'

In [6]:
"""
for investment_id in tqdm(investment_id_ls, desc = 'Progress Bar: Creating Files'):    
    for x in range(start,back_window,file_step_size):        
        save_filename = create_df_panel_parquet(investment_id,(x+1),(x+file_step_size),increment,timestamp_col)
"""

"\nfor investment_id in tqdm(investment_id_ls, desc = 'Progress Bar: Creating Files'):    \n    for x in range(start,back_window,file_step_size):        \n        save_filename = create_df_panel_parquet(investment_id,(x+1),(x+file_step_size),increment,timestamp_col)\n"

In [7]:
"""
%%time
first_flag = 1
for filename in fileList:
    print(filename)
    df = pd.read_parquet(filename, engine='pyarrow')
    
    if first_flag == 1:
        final = df.copy()
        first_flag = 0
    else:
        df = df.drop(['Asset_ID','Count','Open','High','Low','Close','Volume','VWAP','Target'],axis=1,inplace=False)
        final = pd.merge(final, df, left_on='timestamp', right_on='timestamp')
final = final.sort_values(by=['timestamp'])
"""

"\n%%time\nfirst_flag = 1\nfor filename in fileList:\n    print(filename)\n    df = pd.read_parquet(filename, engine='pyarrow')\n    \n    if first_flag == 1:\n        final = df.copy()\n        first_flag = 0\n    else:\n        df = df.drop(['Asset_ID','Count','Open','High','Low','Close','Volume','VWAP','Target'],axis=1,inplace=False)\n        final = pd.merge(final, df, left_on='timestamp', right_on='timestamp')\nfinal = final.sort_values(by=['timestamp'])\n"